0) Carga de datos, librerias y funciones

In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import LatentDirichletAllocation as LDA
import matplotlib.pyplot as plt

In [4]:
ruta='data/resultados_enquestes.csv'
df=pd.read_csv(ruta,sep=',')
df.head(5)


,ID,ENQUESTA_ID,SEMESTRE,PREGUNTA_ID,RESPOSTA_ID,RESPOSTA,IDP,SUMATORI,TIPUS_PREGUNTA,ENQUESTA_CREATED_AT,PREGUNTA_CREATED_AT,RESPOSTA_CREATED_AT,CREATION_DATE,UPDATE_DATE
0,8774167,668,20232.0,17675,426985,SolicitÃ© un cambio de idioma de docencia y de...,1680015,1,PREGUNTA_OBERTA,2024-03-01 09:14:33,2024-03-01 09:14:40,2024-03-27 16:04:38,2024-08-09 09:49:23.133000,2024-08-09 09:49:23.133000
1,8774168,670,NaN,17738,427070,MejorarÃ­a la velocidad de respuesta por parte...,1265565,1,PREGUNTA_OBERTA,2024-05-22 09:02:23,2024-05-22 09:02:32,2024-06-12 17:38:19,2024-08-09 09:49:23.133000,2024-08-09 09:49:23.133000
2,8774170,670,NaN,17738,427313,Algunas asignaturas estÃ¡n en inglÃ©s porque s...,1418238,1,PREGUNTA_OBERTA,2024-05-22 09:02:23,2024-05-22 09:02:32,2024-06-12 19:49:45,2024-08-09 09:49:23.133000,2024-08-09 09:49:23.133000
3,8774171,670,NaN,17738,427349,A Canvas resulta poc intuÃ¯tiu que els recurso...,1414112,1,PREGUNTA_OBERTA,2024-05-22 09:02:23,2024-05-22 09:02:32,2024-06-12 20:10:34,2024-08-09 09:49:23.133000,2024-08-09 09:49:23.133000
4,8774172,670,NaN,17738,427421,"Se necesitan mas profesores vocacionales, y un...",815659,1,PREGUNTA_OBERTA,2024-05-22 09:02:23,2024-05-22 09:02:32,2024-06-12 20:52:45,2024-08-09 09:49:23.133000,2024-08-09 09:49:23.133000


1) Preprocesado
   - Corpus
   - Tokens
   - Stopwords
   - 

In [30]:
import nltk
nltk.download('punkt', download_dir='nltk_data')
from nltk.corpus import PlaintextCorpusReader
from nltk.tokenize import word_tokenize




[nltk_data] Downloading package punkt to nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [16]:
text_column_name ='RESPOSTA'
text_data = df[text_column_name].tolist()
print(type(text_data))
text_data

<class 'list'>


['SolicitÃ© un cambio de idioma de docencia y despuÃ©s de una semana sigo esperando que me lo hagan efectivo teniendo proximas entregas',
 'MejorarÃ\xada la velocidad de respuesta por parte de los profesores responsables de la asignatura que han tardado incluso 1 semana en responder.  Se tendrÃ\xada que tener la posibilidad de tener alguna reuniÃ³n tipo meeting con ellos para resolver dudas y un feedback personalizado que permita saber quÃ© mejorar en concreto.',
 'Algunas asignaturas estÃ¡n en inglÃ©s porque si, cuando yo escogÃ\xad idioma al hacer la matrÃ\xadcula marquÃ© castellano, esta es Sistemas Distribuidos y no se traduce al castellano porque a los tutores no les da la gana. Muy poco satisfecho con esa asignatura en concreto, el resto bien.',
 'A Canvas resulta poc intuÃ¯tiu que els recursos queden desprÃ©s de la tasca. A l&#039;assignatura de InferÃ¨ncia EstadÃ\xadstica no hi ha retroacciÃ³ i nomÃ©s hi ha dues tasques. Ã\x89s l&#039;Ãºnica assignatura que no recomanaria i amb

In [27]:
import re
import html
import unicodedata
def clean_text(text):
    if not isinstance(text, str):
        return ''  # O alguna otra forma de manejar los elementos no cadenas
    
    # Decodificar HTML si es necesario (para manejar caracteres especiales)
    text = html.unescape(text)
    
    # Convertir a minúsculas
    text = text.lower()
    
    # Normalizar y eliminar acentos
    text = unicodedata.normalize('NFD', text)
    text = ''.join(c for c in text if not unicodedata.combining(c))
    
    # Eliminar URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # Eliminar menciones (@usuario) y hashtags
    text = re.sub(r'\@\w+|\#', '', text)
    
    # Eliminar signos de puntuación
    text = re.sub(r'[^\w\s]', '', text)
    
    # Eliminar números si no son necesarios
    text = re.sub(r'\d+', '', text)
    
    # Eliminar espacios en blanco extras
    text = text.strip()
    
    return text

cleaned_data = [clean_text(doc) for doc in text_data]

In [28]:
cleaned_data

['solicita un cambio de idioma de docencia y despuas de una semana sigo esperando que me lo hagan efectivo teniendo proximas entregas',
 'mejoraraa la velocidad de respuesta por parte de los profesores responsables de la asignatura que han tardado incluso  semana en responder  se tendraa que tener la posibilidad de tener alguna reunia³n tipo meeting con ellos para resolver dudas y un feedback personalizado que permita saber qua mejorar en concreto',
 'algunas asignaturas estan en inglas porque si cuando yo escoga idioma al hacer la matracula marqua castellano esta es sistemas distribuidos y no se traduce al castellano porque a los tutores no les da la gana muy poco satisfecho con esa asignatura en concreto el resto bien',
 'a canvas resulta poc intuatiu que els recursos queden despras de la tasca a lassignatura de inferancia estadastica no hi ha retroaccia³ i nomas hi ha dues tasques as laºnica assignatura que no recomanaria i amb la qual estic insatisfet la resta de docents sa³n molt 

In [32]:
nltk.data.path.append('C:\\Users\\asanchezbelb\\Desktop\\UOC\\XaviNPL\\nltk_data')
# Verificar que el recurso 'punkt' está disponible
try:
    nltk.data.find('tokenizers/punkt')
    print("Recurso 'punkt' está disponible.")
except LookupError:
    print("Recurso 'punkt' no está disponible.")


Recurso 'punkt' está disponible.


In [35]:
import spacy

# Cargar los modelos de idioma
nlp_es = spacy.load("es_core_news_sm")
nlp_en = spacy.load("en_core_web_sm")
nlp_ca = spacy.load("ca_core_news_sm")


In [39]:
# Imprimir las primeras 5 entradas de cleaned_data para verificar su estructura
print(cleaned_data[:5])


['solicita un cambio de idioma de docencia y despuas de una semana sigo esperando que me lo hagan efectivo teniendo proximas entregas', 'mejoraraa la velocidad de respuesta por parte de los profesores responsables de la asignatura que han tardado incluso  semana en responder  se tendraa que tener la posibilidad de tener alguna reunia³n tipo meeting con ellos para resolver dudas y un feedback personalizado que permita saber qua mejorar en concreto', 'algunas asignaturas estan en inglas porque si cuando yo escoga idioma al hacer la matracula marqua castellano esta es sistemas distribuidos y no se traduce al castellano porque a los tutores no les da la gana muy poco satisfecho con esa asignatura en concreto el resto bien', 'a canvas resulta poc intuatiu que els recursos queden despras de la tasca a lassignatura de inferancia estadastica no hi ha retroaccia³ i nomas hi ha dues tasques as laºnica assignatura que no recomanaria i amb la qual estic insatisfet la resta de docents sa³n molt bon

In [41]:
def tokenize_text(text):
    # Procesar el texto
    doc = nlp_es(text)
    
    # Extraer tokens
    tokens = [token.text for token in doc]
    
    return tokens
    
    # Procesar el texto
    doc = nlp(text)
    
    # Extraer tokens
    tokens = [token.text for token in doc]
    
    return tokens

tokenized_data = [tokenize_text(doc) for doc in cleaned_data]

# Imprimir los resultados
for idx, tokens in enumerate(tokenized_data):
    print(f"Documento {idx+1}: {tokens}")

In [34]:
tokenized_data = [word_tokenize(doc.lower()) for doc in cleaned_data]
tokenized_data

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\asanchezbelb/nltk_data'
    - 'c:\\Users\\asanchezbelb\\.conda\\envs\\hola\\nltk_data'
    - 'c:\\Users\\asanchezbelb\\.conda\\envs\\hola\\share\\nltk_data'
    - 'c:\\Users\\asanchezbelb\\.conda\\envs\\hola\\lib\\nltk_data'
    - 'C:\\Users\\asanchezbelb\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - 'C:\\Users\\asanchezbelb\\Desktop\\UOC\\XaviNPL\\nltk_data'
**********************************************************************


In [ ]:
for idx, tokens in enumerate(tokenized_data):
    print(f"Documento {idx+1}: {tokens}")

In [ ]:
# Importar las librerías necesarias
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import LatentDirichletAllocation as LDA
import matplotlib.pyplot as plt

# Leer el archivo CSV con los datos
base = pd.read_csv("G:/Unidades compartidas/Data Science Lab/JIRA_2024/UOCLABS-255_Dalilis_Comentarios/Análisis/01-DA-AARON/0.data/df_sentimiento_completo_sin_duplicados.csv", sep=",")
print(base)

# Definir la columna de texto
text_column_name = "text_x"
text_data = base[text_column_name]

# Crear una matriz de términos con CountVectorizer (el equivalente a 'tokens' y 'dfm' en R)
vectorizer = CountVectorizer(stop_words='spanish', token_pattern=r'\b[a-zA-Z]+\b', min_df=10)
dtm = vectorizer.fit_transform(text_data)

# Dividir los datos en entrenamiento (75%) y prueba (25%)
dtm_train, dtm_test = train_test_split(dtm, test_size=0.25, random_state=42)

# Crear el modelo LDA con 7 temas
lda = LDA(n_components=7, random_state=42)
lda.fit(dtm_train)

# Calcular la perplexidad en el conjunto de prueba
perplexity_train = lda.perplexity(dtm_test)
print(perplexity_train)

# Definir un rango de valores de k para evaluar diferentes números de temas
k_values = [2, 4, 8, 16, 32, 64, 128]
perplexities = []

# Bucle para calcular la perplexidad para cada valor de k
for k in k_values:
    print(k)
    lda = LDA(n_components=k, random_state=42)
    lda.fit(dtm_train)
    perplexity = lda.perplexity(dtm_test)
    perplexities.append(perplexity)

# Graficar la perplexidad para diferentes valores de k
plt.plot(k_values, perplexities, marker='o')
plt.xlabel('Número de temas (k)')
plt.ylabel('Perplejidad')
plt.title('Perplejidad para diferentes valores de k')
plt.show()

# Según la gráfica, elegir k=8 para el modelo final
lda_8 = LDA(n_components=8, random_state=42)
lda_8.fit(dtm)

# Obtener la distribución de temas para el primer documento
doc_topic_distribution = lda_8.transform(dtm[0])
print(doc_topic_distribution)

# Modelo con 15 temas para comparar
lda_15 = LDA(n_components=15, random_state=42)
lda_15.fit(dtm)

# Obtener la distribución de temas para el primer documento en el modelo con 15 temas
doc_topic_distribution_15 = lda_15.transform(dtm[0])
print(doc_topic_distribution_15)

# Obtener las palabras principales para cada tema (top terms)
terms = vectorizer.get_feature_names_out()
topic_words = {}
for topic_idx, topic in enumerate(lda_8.components_):
    top_terms_idx = topic.argsort()[-40:][::-1]  # Obtener los índices de las 40 palabras más importantes
    top_terms = [terms[i] for i in top_terms_idx]
    topic_words[topic_idx] = top_terms

print(topic_words)

# Guardar el objeto 'topic_words' en un archivo llamado "tw8_12_data.pkl"
import pickle
with open("tw8_12_data.pkl", "wb") as f:
    pickle.dump(topic_words, f)


In [1]:
import pandas as pd

In [9]:
ruta='data/resultados_enquestes.csv'
df=pd.read_csv(ruta,sep=',')
df


,ID,ENQUESTA_ID,SEMESTRE,PREGUNTA_ID,RESPOSTA_ID,RESPOSTA,IDP,SUMATORI,TIPUS_PREGUNTA,ENQUESTA_CREATED_AT,PREGUNTA_CREATED_AT,RESPOSTA_CREATED_AT,CREATION_DATE,UPDATE_DATE
0,8774167,668,20232.0,17675,426985,SolicitÃ© un cambio de idioma de docencia y de...,1680015,1,PREGUNTA_OBERTA,2024-03-01 09:14:33,2024-03-01 09:14:40,2024-03-27 16:04:38,2024-08-09 09:49:23.133000,2024-08-09 09:49:23.133000
1,8774168,670,NaN,17738,427070,MejorarÃ­a la velocidad de respuesta por parte...,1265565,1,PREGUNTA_OBERTA,2024-05-22 09:02:23,2024-05-22 09:02:32,2024-06-12 17:38:19,2024-08-09 09:49:23.133000,2024-08-09 09:49:23.133000
2,8774170,670,NaN,17738,427313,Algunas asignaturas estÃ¡n en inglÃ©s porque s...,1418238,1,PREGUNTA_OBERTA,2024-05-22 09:02:23,2024-05-22 09:02:32,2024-06-12 19:49:45,2024-08-09 09:49:23.133000,2024-08-09 09:49:23.133000
3,8774171,670,NaN,17738,427349,A Canvas resulta poc intuÃ¯tiu que els recurso...,1414112,1,PREGUNTA_OBERTA,2024-05-22 09:02:23,2024-05-22 09:02:32,2024-06-12 20:10:34,2024-08-09 09:49:23.133000,2024-08-09 09:49:23.133000
4,8774172,670,NaN,17738,427421,"Se necesitan mas profesores vocacionales, y un...",815659,1,PREGUNTA_OBERTA,2024-05-22 09:02:23,2024-05-22 09:02:32,2024-06-12 20:52:45,2024-08-09 09:49:23.133000,2024-08-09 09:49:23.133000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2586,8796078,670,NaN,17738,431426,ValorarÃ­a positivamente tener vÃ­as de contac...,663572,1,PREGUNTA_OBERTA,2024-05-22 09:02:23,2024-05-22 09:02:32,2024-06-27 18:37:51,2024-08-09 09:49:23.133000,2024-08-09 09:49:23.133000
2587,8796080,670,NaN,17738,431487,Me parece poco acierto que no nos den otra con...,1183752,1,PREGUNTA_OBERTA,2024-05-22 09:02:23,2024-05-22 09:02:32,2024-06-28 00:30:32,2024-08-09 09:49:23.133000,2024-08-09 09:49:23.133000
2588,8796085,670,NaN,17738,431692,Seria necesario incorporar atenciÃ³n telefÃ³nica.,970740,1,PREGUNTA_OBERTA,2024-05-22 09:02:23,2024-05-22 09:02:32,2024-06-29 14:17:45,2024-08-09 09:49:23.133000,2024-08-09 09:49:23.133000
2589,8796086,670,NaN,17738,431779,Valoro positivament estudiar a la UOC pel fet ...,1416231,1,PREGUNTA_OBERTA,2024-05-22 09:02:23,2024-05-22 09:02:32,2024-06-30 16:38:51,2024-08-09 09:49:23.133000,2024-08-09 09:49:23.133000
